In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from warnings import filterwarnings
from sklearn.ensemble import GradientBoostingRegressor
filterwarnings('ignore')

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Hitters.csv to Hitters.csv
User uploaded file "Hitters.csv" with length 20906 bytes


In [3]:
hit = pd.read_csv("Hitters.csv")
df = hit.copy()
df = df.dropna()
dms = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
y = df["Salary"]
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=42)


In [4]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.7 MB/s eta 0:00:00


In [5]:
from catboost import CatBoostRegressor

In [8]:
catb = CatBoostRegressor()
catb_model = catb.fit(X_train, y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 3.69ms	remaining: 3.69s
1:	learn: 431.3923642	total: 5.54ms	remaining: 2.76s
2:	learn: 424.8820360	total: 7.79ms	remaining: 2.59s
3:	learn: 418.2514904	total: 10.2ms	remaining: 2.54s
4:	learn: 412.6394021	total: 12.3ms	remaining: 2.45s
5:	learn: 406.6247020	total: 14.4ms	remaining: 2.38s
6:	learn: 400.5321206	total: 16.5ms	remaining: 2.35s
7:	learn: 394.6683437	total: 18.6ms	remaining: 2.3s
8:	learn: 388.2496484	total: 20.7ms	remaining: 2.28s
9:	learn: 382.9448842	total: 23ms	remaining: 2.27s
10:	learn: 377.2600080	total: 25.1ms	remaining: 2.25s
11:	learn: 372.4829606	total: 27.4ms	remaining: 2.26s
12:	learn: 366.6823437	total: 29.5ms	remaining: 2.24s
13:	learn: 362.6076230	total: 31.6ms	remaining: 2.23s
14:	learn: 358.0107745	total: 33.8ms	remaining: 2.22s
15:	learn: 353.2802665	total: 35.9ms	remaining: 2.21s
16:	learn: 348.5646265	total: 38ms	remaining: 2.2s
17:	learn: 343.6407912	total: 40.2ms	remaining: 2.19s
18:	learn: 339

In [9]:
y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

351.194631344607

In [10]:
catb_grid = {'depth': [3,4,5,6,7,8],
             'iterations' : [200,500,1000,2000],
            'learning_rate': [0.01, 0.1, 0.5, 1],
              }

In [13]:
catb = CatBoostRegressor()
catb_cv_model = GridSearchCV(catb, catb_grid, cv= 10, n_jobs = -1, verbose = 2)

In [ ]:
catb_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 96 candidates, totalling 960 fits
0:	learn: 425.7900818	total: 3.4ms	remaining: 677ms
1:	learn: 404.8723520	total: 5.2ms	remaining: 515ms
2:	learn: 387.4057666	total: 6.11ms	remaining: 401ms
3:	learn: 372.2801584	total: 7.46ms	remaining: 366ms
4:	learn: 358.9204229	total: 9.36ms	remaining: 365ms
5:	learn: 347.0083933	total: 10.8ms	remaining: 350ms
6:	learn: 336.0130818	total: 12.5ms	remaining: 346ms
7:	learn: 324.3923300	total: 14.4ms	remaining: 346ms
8:	learn: 314.8690957	total: 15.7ms	remaining: 333ms
9:	learn: 308.5075563	total: 17.5ms	remaining: 332ms
10:	learn: 298.8587285	total: 19.8ms	remaining: 339ms
11:	learn: 294.7655438	total: 21ms	remaining: 329ms
12:	learn: 288.0697862	total: 21.6ms	remaining: 311ms
13:	learn: 282.6697154	total: 22.2ms	remaining: 295ms
14:	learn: 277.6121667	total: 22.9ms	remaining: 282ms
15:	learn: 273.4383979	total: 23.9ms	remaining: 275ms
16:	learn: 269.1556201	total: 24.8ms	remaining: 267ms
17:	learn: 264.8098704	total: 26.

GridSearchCV(cv=10,
             estimator=<catboost.core.CatBoostRegressor object at 0x7f0b0d1d5de0>,
             n_jobs=-1,
             param_grid={'depth': [3, 4, 5, 6, 7, 8],
                         'iterations': [200, 500, 1000, 2000],
                         'learning_rate': [0.01, 0.1, 0.5, 1]},
             verbose=2)

In [ ]:
catb_tuned = CatBoostRegressor(iterations = 200,
                               learning_rate = 0.01,
                               depth = 8)

catb_tuned = catb_tuned.fit(X_train, y_train)

In [ ]:
y_pred = catb_tuned.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))